<a href="https://colab.research.google.com/github/lorifin/PAL/blob/main/Codir_kpi_rapport_git_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Installer les dépendances (une seule fois)
!pip install openai matplotlib pandas slack_sdk fpdf

# 2. Imports
import pandas as pd
import matplotlib.pyplot as plt
import openai
from slack_sdk import WebClient
from fpdf import FPDF
from datetime import datetime

# 3. Télécharger la police Unicode pour le PDF
!wget https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/DejaVuSans.ttf -O DejaVuSans.ttf

# 4. Paramètres API (REMPLACE par tes vraies clés avant usage)
OPENAI_API_KEY = ""                    # Ta clé OpenAI               # Ton token Slack
SLACK_CHANNEL_ID = ""             # ID du canal Slack
SLACK_BOT_TOKEN = ""             # Bot User OAuth Token Slack
SLACK_CHANNEL = ""

# 5. Charger les données depuis le CSV
df = pd.read_csv("data_codir.csv")
display(df)

# 6. Générer et sauvegarder le graphique
mois = df["Mois"]
ca = df["CA réalisé (€)"]
objectif = df["Objectif (€)"]
pipe = df["Pipe (€)"]

plt.figure(figsize=(8,5))
plt.plot(mois, ca, marker='o', label='CA Réalisé')
plt.plot(mois, objectif, marker='o', linestyle='--', label='Objectif')
plt.bar(mois, pipe, alpha=0.2, label='Pipe')
plt.title("Synthèse Commerciale Trimestrielle")
plt.ylabel("Montant (€)")
plt.legend()
plt.tight_layout()
plt.savefig("ventes_graph.png")
plt.close()

# 7. Générer la synthèse Codir avec GPT
openai.api_key = OPENAI_API_KEY

prompt = f"""
Voici le tableau des indicateurs commerciaux du trimestre :
Mois : {', '.join(df['Mois'])}
CA réalisé : {', '.join(str(x) for x in df['CA réalisé (€)'])}
Objectif : {', '.join(str(x) for x in df['Objectif (€)'])}
Pipe : {', '.join(str(x) for x in df['Pipe (€)'])}
Opportunités : {', '.join(df['Opportunités'])}
Risques : {', '.join(df['Risques/Blocages'])}

Donne-moi :
1. Une synthèse Codir (en 3 phrases)
2. Les 2 points de vigilance et opportunités majeures
3. 2 actions prioritaires à piloter ce trimestre
"""

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=400,
    temperature=0.2,
)
synthese_codir = response.choices[0].message.content
print("==== Synthèse automatique Codir ====")
print(synthese_codir)


import os
if os.path.exists("DejaVuSans.ttf"):
    os.remove("DejaVuSans.ttf")
!wget -O DejaVuSans.ttf https://github.com/dejavu-fonts/dejavu-fonts/raw/master/ttf/DejaVuSans.ttf



# 8. Générer le PDF avec la police Unicode


# Installez fpdf2 : pip install fpdf2
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', '', 12)
pdf.cell(200, 10, txt="Synthèse Codir trimestrielle", ln=1, align='C')
pdf.ln(10)
pdf.multi_cell(0, 10, synthese_codir)  # fpdf2 gère mieux l'Unicode
pdf.output("rapport_codir.pdf")
print("✅ Rapport PDF généré.")



# 9. Config Slack client
slack_client = WebClient(token=SLACK_BOT_TOKEN)

# 10. Envoi de la synthèse Codir (texte)
try:
    slack_client.chat_postMessage(
        channel=SLACK_CHANNEL_ID,
        text=f"Synthèse Codir automatique :\n{synthese_codir}"
    )
    print("✅ Synthèse envoyée sur Slack.")
except Exception as e:
    print(f"❌ Erreur Slack (texte) : {e}")

# 11. Envoi du graphique (image)
try:
    slack_client.files_upload_v2(
        channel=SLACK_CHANNEL_ID,
        file="ventes_graph.png",
        title="Graphique Codir",
        initial_comment="Voici l’évolution des ventes 📈"
    )
    print("✅ Graphe envoyé sur Slack.")
except Exception as e:
    print(f"❌ Erreur Slack (graphe) : {e}")

# 12. Envoi du PDF (rapport Codir)
try:
    slack_client.files_upload_v2(
        channel=SLACK_CHANNEL_ID,
        file="rapport_codir.pdf",
        title="Rapport PDF Codir",
        initial_comment="Voici le rapport Codir PDF du trimestre 📄"
    )
    print("✅ PDF envoyé sur Slack.")
except Exception as e:
    print(f"❌ Erreur Slack (PDF) : {e}")
